# Train A Shape Classifier Model



In [ ]:
import json
import os
import pandas as pd

def load_classification_dataset(dataset_dir):
    """
    Load a classification dataset from a directory layout where each subdirectory
    represents a class, and each image in the subdirectory belongs to that class.

    Args:
        dataset_dir (str): Path to the dataset directory.

    Returns:
        pd.DataFrame: DataFrame containing two columns: 'filename' and 'label'.
    """
    data = []

    # Traverse through each class folder in the dataset directory
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)

        # Ensure that it's a directory (not a file or other item)
        if os.path.isdir(class_dir):
            # Get all image files in the class directory
            for img_file in os.listdir(class_dir):
                if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Filter image files by extension
                    img_path = os.path.join(class_name, img_file)
                    data.append([img_path, class_name])

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data, columns=['filename', 'label'])
    return df

# Example usage
df_train = load_classification_dataset("datasets/train")
df_train["split"] = "train"
df_test = load_classification_dataset("datasets/test")
df_test["split"] = "test"
df = pd.concat([df_train, df_test])
df

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms